In [ ]:
import pandas as pd

# Load the uploaded datasets
geo_lookup_df = pd.read_csv('/mnt/data/geo_lookup_v2.csv')
orders_df = pd.read_csv('/mnt/data/orders_v2.csv')
order_status_df = pd.read_csv('/mnt/data/order_status_v2.csv')
customers_df = pd.read_csv('/mnt/data/customers_v2.csv')

In [ ]:
# Merge customers with orders on `customer_id` to enrich orders with customer data
orders_with_customers = orders_df.merge(customers_df, left_on='customer_id', right_on='id', how='left')

# Merge with order_status on `id` and `order_id` to include order status details
orders_with_status = orders_with_customers.merge(order_status_df, left_on='id_x', right_on='order_id', how='left')

# Merge with geo_lookup on `country_code` and `country` to append regional data
final_merged_df = orders_with_status.merge(geo_lookup_df, left_on='country_code', right_on='country', how='left')

# Inspect the merged dataframe structure
final_merged_df.head()

In [ ]:
import ace_tools as tools  # Re-importing to fix the issue

# Display the pivot table to the user again
tools.display_dataframe_to_user(name="Sales Performance Pivot Table", dataframe=pivot_data)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a new column for monthly analysis
final_merged_df['month'] = pd.to_datetime(final_merged_df['purchase_ts_x']).dt.month

# Monthly aggregation
monthly_data = final_merged_df.groupby(['year', 'month']).agg(
    sales=('usd_price', 'sum'),
    order_count=('id_x', 'count'),
    AOV=('usd_price', 'mean')
).reset_index()

# Calculate growth rates for monthly data
monthly_data['sales_growth_%'] = monthly_data.groupby('year')['sales'].pct_change() * 100
monthly_data['AOV_growth_%'] = monthly_data.groupby('year')['AOV'].pct_change() * 100
monthly_data['order_count_growth_%'] = monthly_data.groupby('year')['order_count'].pct_change() * 100

# Set up the plotting area
fig, axs = plt.subplots(3, 2, figsize=(16, 12))

# Line plots for monthly metrics by year
sns.lineplot(data=monthly_data, x='month', y='sales', hue='year', ax=axs[0, 0])
axs[0, 0].set_title("Total Sales by Month")
axs[0, 0].set_ylabel("Total Sales (USD)")
axs[0, 0].legend(title="Year")

sns.lineplot(data=monthly_data, x='month', y='AOV', hue='year', ax=axs[0, 1])
axs[0, 1].set_title("Average Order Value (AOV) by Month")
axs[0, 1].set_ylabel("Average Order Value (USD)")
axs[0, 1].legend(title="Year")

sns.lineplot(data=monthly_data, x='month', y='order_count', hue='year', ax=axs[1, 0])
axs[1, 0].set_title("Total Orders by Month")
axs[1, 0].set_ylabel("Total Orders")
axs[1, 0].legend(title="Year")

# Pivot data for heatmaps
sales_growth_pivot = monthly_data.pivot("year", "month", "sales_growth_%")
AOV_growth_pivot = monthly_data.pivot("year", "month", "AOV_growth_%")
order_growth_pivot = monthly_data.pivot("year", "month", "order_count_growth_%")

# Heatmaps for growth rates
sns.heatmap(sales_growth_pivot, ax=axs[1, 1], annot=True, fmt=".1f", cmap="Blues", cbar=False)
axs[1, 1].set_title("Sales Growth Rate (%)")
axs[1, 1].set_xlabel("Month")
axs[1, 1].set_ylabel("Year")

sns.heatmap(AOV_growth_pivot, ax=axs[2, 0], annot=True, fmt=".1f", cmap="Blues", cbar=False)
axs[2, 0].set_title("AOV Growth Rate (%)")
axs[2, 0].set_xlabel("Month")
axs[2, 0].set_ylabel("Year")

sns.heatmap(order_growth_pivot, ax=axs[2, 1], annot=True, fmt=".1f", cmap="Blues", cbar=False)
axs[2, 1].set_title("Order Growth Rate (%)")
axs[2, 1].set_xlabel("Month")
axs[2, 1].set_ylabel("Year")

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
# Set up the plotting area with 3 columns and 2 rows
fig, axs = plt.subplots(2, 3, figsize=(18, 10))

# Line plots for monthly metrics by year
sns.lineplot(data=monthly_data, x='month', y='sales', hue='year', ax=axs[0, 0])
axs[0, 0].set_title("Total Sales by Month")
axs[0, 0].set_ylabel("Total Sales (USD)")
axs[0, 0].set_xlabel("Month")
axs[0, 0].legend(title="Year")

sns.lineplot(data=monthly_data, x='month', y='AOV', hue='year', ax=axs[0, 1])
axs[0, 1].set_title("Average Order Value (AOV) by Month")
axs[0, 1].set_ylabel("Average Order Value (USD)")
axs[0, 1].set_xlabel("Month")
axs[0, 1].legend(title="Year")

sns.lineplot(data=monthly_data, x='month', y='order_count', hue='year', ax=axs[0, 2])
axs[0, 2].set_title("Total Orders by Month")
axs[0, 2].set_ylabel("Total Orders")
axs[0, 2].set_xlabel("Month")
axs[0, 2].legend(title="Year")

# Heatmaps for growth rates
sns.heatmap(sales_growth_pivot, ax=axs[1, 0], annot=True, fmt=".1f", cmap="Blues", cbar=False)
axs[1, 0].set_title("Sales Growth Rate (%)")
axs[1, 0].set_xlabel("Month")
axs[1, 0].set_ylabel("Year")

sns.heatmap(AOV_growth_pivot, ax=axs[1, 1], annot=True, fmt=".1f", cmap="Blues", cbar=False)
axs[1, 1].set_title("AOV Growth Rate (%)")
axs[1, 1].set_xlabel("Month")
axs[1, 1].set_ylabel("Year")

sns.heatmap(order_growth_pivot, ax=axs[1, 2], annot=True, fmt=".1f", cmap="Blues", cbar=False)
axs[1, 2].set_title("Order Growth Rate (%)")
axs[1, 2].set_xlabel("Month")
axs[1, 2].set_ylabel("Year")

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
#Product performance metrics

# Calculate product-level metrics
product_metrics = final_merged_df.groupby('product_name').agg(
    total_revenue=('usd_price', 'sum'),
    order_count=('id_x', 'count'),
    AOV=('usd_price', 'mean'),
    refund_rate=('refund_ts', lambda x: x.notnull().sum() / len(x) * 100)
).reset_index()

# Add percentage columns
product_metrics['total_revenue_%'] = product_metrics['total_revenue'] / product_metrics['total_revenue'].sum() * 100
product_metrics['order_count_%'] = product_metrics['order_count'] / product_metrics['order_count'].sum() * 100

# Add a grand total row
grand_total = {
    'product_name': 'Grand Total',
    'total_revenue': product_metrics['total_revenue'].sum(),
    'order_count': product_metrics['order_count'].sum(),
    'AOV': product_metrics['total_revenue'].sum() / product_metrics['order_count'].sum(),
    'refund_rate': (final_merged_df['refund_ts'].notnull().sum() / len(final_merged_df)) * 100,
    'total_revenue_%': 100,
    'order_count_%': 100
}

product_metrics = pd.concat([product_metrics, pd.DataFrame([grand_total])], ignore_index=True)

# Format table for display
product_metrics['total_revenue'] = product_metrics['total_revenue'].apply(lambda x: f"${x:,.0f}")
product_metrics['total_revenue_%'] = product_metrics['total_revenue_%'].apply(lambda x: f"{x:.0f}%")
product_metrics['AOV'] = product_metrics['AOV'].apply(lambda x: f"${x:,.0f}")
product_metrics['order_count'] = product_metrics['order_count'].apply(lambda x: f"{x:,}")
product_metrics['order_count_%'] = product_metrics['order_count_%'].apply(lambda x: f"{x:.0f}%")
product_metrics['refund_rate'] = product_metrics['refund_rate'].apply(lambda x: f"{x:.0f}%")

# Display the formatted table to the user
import ace_tools as tools; tools.display_dataframe_to_user(name="Product Performance Metrics", dataframe=product_metrics)

In [ ]:
# Yearly customer metrics
final_merged_df['purchase_year'] = pd.to_datetime(final_merged_df['purchase_ts_x']).dt.year

customer_metrics = final_merged_df.groupby('purchase_year').agg(
    unique_customers=('customer_id', 'nunique'),
    repeat_customers=('customer_id', lambda x: x[x.duplicated()].nunique())
).reset_index()

# Calculate repeat rate
customer_metrics['repeat_rate'] = (customer_metrics['repeat_customers'] / customer_metrics['unique_customers']) * 100

# Format table for display
tools.display_dataframe_to_user(name="Yearly Customer Metrics", dataframe=customer_metrics)

# Provide insights
customer_metrics

In [ ]:
# Loyalty vs non-loyalty program
loyalty_metrics = final_merged_df.groupby(['purchase_year', 'loyalty_program']).agg(
    sales=('usd_price', 'sum'),
    order_count=('id_x', 'count'),
    AOV=('usd_price', 'mean')
).reset_index()

# Add sales growth, AOV growth, and order count growth
loyalty_metrics['sales_growth_%'] = loyalty_metrics.groupby('loyalty_program')['sales'].pct_change() * 100
loyalty_metrics['AOV_growth_%'] = loyalty_metrics.groupby('loyalty_program')['AOV'].pct_change() * 100
loyalty_metrics['order_growth_%'] = loyalty_metrics.groupby('loyalty_program')['order_count'].pct_change() * 100

# Pivot for table display
pivot_loyalty = loyalty_metrics.pivot(index='purchase_year', columns='loyalty_program', 
                                      values=['sales', 'sales_growth_%', 'AOV', 'AOV_growth_%', 'order_count', 'order_growth_%'])

# Add grand totals for each metric
grand_totals = {
    ('sales', 0): loyalty_metrics[loyalty_metrics['loyalty_program'] == 0]['sales'].sum(),
    ('sales', 1): loyalty_metrics[loyalty_metrics['loyalty_program'] == 1]['sales'].sum(),
    ('order_count', 0): loyalty_metrics[loyalty_metrics['loyalty_program'] == 0]['order_count'].sum(),
    ('order_count', 1): loyalty_metrics[loyalty_metrics['loyalty_program'] == 1]['order_count'].sum(),
    ('AOV', 0): loyalty_metrics[loyalty_metrics['loyalty_program'] == 0]['sales'].sum() / loyalty_metrics[loyalty_metrics['loyalty_program'] == 0]['order_count'].sum(),
    ('AOV', 1): loyalty_metrics[loyalty_metrics['loyalty_program'] == 1]['sales'].sum() / loyalty_metrics[loyalty_metrics['loyalty_program'] == 1]['order_count'].sum()
}

pivot_loyalty.loc['Grand Total'] = grand_totals

# Reset index for display purposes
pivot_loyalty = pivot_loyalty.reset_index()

# Format for presentation
tools.display_dataframe_to_user(name="Loyalty Program Performance", dataframe=pivot_loyalty)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Create monthly data for plots
final_merged_df['month_year'] = final_merged_df['purchase_ts_x'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m'))
monthly_loyalty = final_merged_df.groupby(['month_year', 'loyalty_program']).agg(
    sales=('usd_price', 'sum'),
    order_count=('id_x', 'count'),
    AOV=('usd_price', 'mean')
).reset_index()

# Plot metrics
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

sns.lineplot(data=monthly_loyalty, x='month_year', y='sales', hue='loyalty_program', ax=axs[0])
axs[0].set_title("Total Revenue by Month")
axs[0].set_ylabel("Total Revenue (USD)")
axs[0].set_xlabel("Month-Year")
axs[0].tick_params(axis='x', rotation=45)

sns.lineplot(data=monthly_loyalty, x='month_year', y='order_count', hue='loyalty_program', ax=axs[1])
axs[1].set_title("Total Orders by Month")
axs[1].set_ylabel("Total Orders")
axs[1].set_xlabel("Month-Year")
axs[1].tick_params(axis='x', rotation=45)

sns.lineplot(data=monthly_loyalty, x='month_year', y='AOV', hue='loyalty_program', ax=axs[2])
axs[2].set_title("Average Order Value (AOV) by Month")
axs[2].set_ylabel("AOV (USD)")
axs[2].set_xlabel("Month-Year")
axs[2].tick_params(axis='x', rotation=45)

# Adjust layout and show plots
plt.tight_layout()
plt.show()